In [3]:
!pip install openai


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!python -m openai.version
import openai
import tqdm
print("done")

<frozen runpy>:128: RuntimeWarning: 'openai.version' found in sys.modules after import of package 'openai', but prior to execution of 'openai.version'; this may result in unpredictable behaviour


done


In [5]:
from dotenv import load_dotenv
import os

import warnings

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('keys.txt')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_BASE_URL = os.getenv('OPENAI_BASE_URL')
print(OPENAI_API_KEY)
print(OPENAI_BASE_URL)

028fa2e1-fb69-4cca-89aa-1e11ffc4dcc1
https://openai.dplit.com/v1


In [ ]:
from openai import OpenAI

client = OpenAI()  # reads env: OPENAI_API_KEY (+ OPENAI_BASE_URL if set)

prompt = """
You are a senior QA engineer.

The following is your assignment:

Feature Request:
"We need dark mode in our app so users can read at night without straining their eyes."

Your task:
1. Create a **Mini Test Plan** with 3–5 test cases (happy path, edge, error).
   Include: ID, Title, Type, Priority, Preconditions, Steps, Expected Result.
2. Create a **Boundary Table** for key fields like theme preference, toggle state, and system setting.
   Columns: Field | Min | Max | Null/Empty | Invalid.
3. Create **1 API Test Case in JSON format** for the Dark Mode toggle API (`/settings/theme`).
   Include: id, method, path, payload, expectedStatus, assertKeys.
4. Output everything in clean **Markdown** format titled:
   `dark_mode_tests_DPL.md`.

Make sure your answer is final, self-contained, and does not ask me for clarification.
"""


resp = client.chat.completions.create(
    model="gpt-4o-mini",            # low-cost, solid default
    messages=[
        {"role": "system", "content": "You are a senior QA engineer who writes precise and concise test documentation."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=200                  # cap output size to control cost
)

output = resp.choices[0].message.content
print(output)

with open("dark_mode_tests_team.md", "w", encoding="utf-8") as f:
    f.write(output)

print("\n✅ File saved as dark_mode_tests_team.md")

```markdown
# dark_mode_tests_DPL.md

## Mini Test Plan

| ID  | Title                          | Type         | Priority | Preconditions                        | Steps                                                                                                                                                  | Expected Result                                                  |
|-----|--------------------------------|--------------|----------|--------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------|
| TC1 | Enable Dark Mode               | Happy Path   | High     | User is logged in and on the settings page | 1. Navigate to settings page.<br>2. Toggle the dark mode switch to 'ON'.                                                                              | Dark mode is activated, and

In [1]:
#!/usr/bin/env python3
"""
Solution Script: Feature Request -> QA Test Assets

Generates:
- Mini test plan (3–5 test cases)
- Boundary table
- One API test case in JSON

Usage:
  python solution_qa_tests.py -f "We need dark mode..."
"""

#!/usr/bin/env python3
"""
Solution Script: Feature Request -> QA Test Assets

Generates Markdown with:
- Mini test plan (3–5 test cases)
- Boundary table
- One API test case in JSON

Usage:
  python solution.py -f "We need dark mode..." -o qa_backlog_solution.md

Requires:
  pip install openai>=1.40.0
  export/setx OPENAI_API_KEY=...
"""

import argparse
import os

# Use a sentinel token <<FEATURE>> and simple .replace() to avoid brace escaping issues
PROMPT = """You are a QA Lead.
From the FEATURE REQUEST, generate concise Markdown with:

### Mini Test Plan
- 3–5 test cases, table format with: ID, Title, Type (happy/edge/error), Priority, Preconditions, Steps, Expected.

### Boundary Table
- Columns: Field | Min | Max | Null/Empty | Invalid

### API Test JSON
- A JSON array with one test object:
  {{
    "id": "TC-API-001",
    "method": "POST",
    "path": "/settings/theme",
    "payload": {{"theme": "dark"}},
    "expectedStatus": 200,
    "assertKeys": ["theme", "appliedAt"]
  }}

FEATURE REQUEST:
<<FEATURE>>
"""

def generate_with_openai(feature: str, model: str = "gpt-4o-mini") -> str:
    try:
        from openai import OpenAI
    except Exception as e:
        raise RuntimeError("OpenAI SDK missing. Run: pip install openai>=1.40.0") from e

    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("Set OPENAI_API_KEY to use this script.")

    client = OpenAI(api_key=api_key)
    prompt = PROMPT.replace("<<FEATURE>>", feature)

    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    return (resp.choices[0].message.content or "").strip()

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("-f", "--feature", required=True, help="Feature request text.")
    parser.add_argument("-o", "--out", default=None, help="Output markdown file path.")
    parser.add_argument("--model", default="gpt-4o-mini", help="OpenAI model to use.")
    args = parser.parse_args()

    md = generate_with_openai(args.feature.strip(), args.model)

    if args.out:
        with open(args.out, "w", encoding="utf-8") as f:
            f.write(md)
        print(f"Wrote output to {args.out}")
    else:
        print(md)

if __name__ == "__main__":
    main()


"""

LINUX
export OPENAI_API_KEY="sk-..."   # Windows: setx OPENAI_API_KEY "sk-..."
WINDOWS
set OPENAI_API_KEY="sk-..."   # Windows: setx OPENAI_API_KEY "sk-..."

python solution.py -f "We need dark mode in our app so users can read at night without straining their eyes." -o qa_backlog_solution.md

"""

RuntimeError: Set OPENAI_API_KEY to use this script.